**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video
!pip install opencv-python

     |████████████████████████████████| 2.3MB 3.8MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# A enlever
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15,7]


Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act selects the next action the agent will take at the state s. There are two modes depending on whether we are in train or in test. In train, an epsilon of 0.1 indicates that 10% of the time, an action is chosen randomly instead of the one given by learned_act. This allows the agent to discover new paths that he would not have considered otherwise. This is the dilemma between eploitation and exploration. In test period, the goal being to maximize the gain, we deactivate this mechanism.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position```: position is an $N \times N$ matrix where the rat position is 1, the allowed position for the future state have the value 0, and the other values are set to -1.

```board``` : the variable board is an $N \times N$ matrix that keeps track of the reward of each cell of the  grid.it's updated each time the rat move to a cell.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):

        return np.random.randint(0, self.n_action, size=1)[0]


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix='', verbose=True):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        

        state = env.reset()

        win = 0
        lose = 0

        while env.t < T :

          action = agent.learned_act(state)

          prev_state = state
          state, reward, game_over = env.act(action)

          if reward > 0:
            win = win + reward
          if reward < 0:
            lose = lose -reward

        #####
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        if verbose : 
          print("Win/lose count {}/{}. Average score ({})"
                .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent_rdm = RandomAgent()

test(agent_rdm,env,1,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/18.0. Average score (-8.5)
Final score: -8.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

1 : 
In the discounted inifinite case, the expression for the $Q$ function is :

\begin{equation*}Q^\pi(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

Because of the Markov property, and putting t' = t-1 :  we can write : 

\begin{equation*}
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p^{\pi}(s',a'|s,a)[\sum_{t'=0}^{\infty}\gamma^{t'}r(s_{t'},a_{t'})|s_{0}=s',a_{0}=a'] \> 
\end{equation*}

\begin{equation*}
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p^{\pi}(s',a'|s,a) Q^{\pi}(s',a')\> 
\end{equation*}

\begin{equation*}
= E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')])\> 
\end{equation*}

2 : 
The optimal $Q$ function is defined as : 

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a)=Q^{\pi^{*}}(s,a) \> .
\end{equation*}

so, we replace it in the last equation of part 1 and we get : 
$$=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{\pi^{*}}(s',a')]$$



3 : 

The optimale $Q$ function is the only function that satisfies the Bellmann equation. Threfore, we want to find the $\theta$ so that the difference :
$$r(s,a)+\gamma\max_{a'}Q^{*}(s',a')-Q^{*}(s,a)$$
is minimal (ideally 0) on average with respect to $p^{\pi^*}$. 
If $Q$ is $\theta$-parametrized, then minimising $\mathcal{L}(\theta)$ will make $Q_\theta$ converge to $Q^*$, with :
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory :
          del self.memory[0]
        self.memory.append(m)

    def random_access(self):
      return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix='',verbose=True):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose


        if verbose : 
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')



***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

***
***

We trained the model for 50 epochs and for different sets of parameters, and used the average score over 10 epochs on the test phase to evaluate it.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.gamma = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):

            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_ 

            # We only take into account the states and actions where we have new information. 
            # In the others, the target is the input, and they do not impact the gradient, thus the update of theta.
            target_q[i] =self.model.predict(np.expand_dims(s_, axis = 0))
            
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_ + self.gamma * np.max(self.model.predict(np.expand_dims(n_s_, axis = 0)))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation = "relu"))
        model.add(Dense(4, activation = 'softmax'))
        
        model.compile(optimizer = 'adam', loss = "mse")
        self.model = model
        

We will try a small model then a big model with Adam, a batch size of 64, a learning rate of 0.2 and an epsilon value of 0.1. We will then pick one and find optimal values for the memory size and learning rate. Let's begin with the first model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 30, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 3.5


In [18]:
HTML(display_videos('fc_test9.mp4'))

Not really good. We can see that the agent gets stuck really fast

Now let's try  a bigger model 


In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation = "relu"))
        model.add(Dense(4, activation = 'softmax'))
        
        model.compile(optimizer = 'adam', loss = "mse")
        self.model = model

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 30, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 4.1


In [21]:
HTML(display_videos('fc_test9.mp4'))

The bigger model seems better. Let's try some other learning rates

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(256, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(128, activation = "relu"))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 30, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 0.2


In [24]:
HTML(display_videos('fc_test9.mp4'))

with 0.01, the agent learn nothing. Let's switch back to 0.1 and test the memory size



In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
train(agent, env, 30, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 3.05


In [27]:
HTML(display_videos('fc_test9.mp4'))

In [28]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=4000, batch_size = 64)
train(agent, env, 30, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 2.45


In [29]:
HTML(display_videos('fc_test9.mp4'))

The first value seems good. let's stick with 2000
we can see that the stucking problem is still there


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
from keras.layers import MaxPool2D
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(25,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation = 'softmax'))
        
        model.compile('adam', "mse")
        self.model = model

In [31]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent,env,30,prefix='cnn_train')

Epoch 001/030 | Loss 0.0016 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/030 | Loss 0.0046 | Win/lose count 6.0/4.0 (2.0)
Epoch 003/030 | Loss 0.0065 | Win/lose count 11.0/3.0 (8.0)
Epoch 004/030 | Loss 0.0066 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/030 | Loss 0.0150 | Win/lose count 6.0/2.0 (4.0)
Epoch 006/030 | Loss 0.0131 | Win/lose count 14.0/5.0 (9.0)
Epoch 007/030 | Loss 0.0064 | Win/lose count 6.5/1.0 (5.5)
Epoch 008/030 | Loss 0.0086 | Win/lose count 2.5/0 (2.5)
Epoch 009/030 | Loss 0.0076 | Win/lose count 11.5/1.0 (10.5)
Epoch 010/030 | Loss 0.0073 | Win/lose count 13.0/4.0 (9.0)
Epoch 011/030 | Loss 0.0078 | Win/lose count 5.5/3.0 (2.5)
Epoch 012/030 | Loss 0.0072 | Win/lose count 1.0/2.0 (-1.0)
Epoch 013/030 | Loss 0.0049 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/030 | Loss 0.0066 | Win/lose count 17.0/10.0 (7.0)
Epoch 015/030 | Loss 0.0164 | Win/lose count 18.0/4.0 (14.0)
Epoch 016/030 | Loss 0.0097 | Win/lose count 14.5/4.0 (10.5)
Epoch 017/030 | Loss 0.0188 | Win/lose count 1

In [32]:
test(agent,env,epochs_test,prefix='cnn_test')

Win/lose count 11.5/0. Average score (11.5)
Win/lose count 6.5/0. Average score (9.0)
Win/lose count 3.5/1.0. Average score (6.833333333333333)
Win/lose count 4.5/0. Average score (6.25)
Win/lose count 16.5/0. Average score (8.3)
Win/lose count 4.0/0. Average score (7.583333333333333)
Win/lose count 4.5/0. Average score (7.142857142857143)
Win/lose count 7.5/0. Average score (7.1875)
Win/lose count 2.5/0. Average score (6.666666666666667)
Win/lose count 8.0/1.0. Average score (6.7)
Final score: 6.7


In [33]:
HTML(display_videos('cnn_test9.mp4'))



the CNN seems way better than the FC network, but the stucking problem is still there let's compare them side by side.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

we will do this for 3 different temperatures values
first : with 0.1![alt text](https://), then with 0.5. we can then compare with previous results that are for 0.3

In [34]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 1.0/0. Average score (1.5)
Win/lose count 4.0/0. Average score (2.3333333333333335)
Win/lose count 1.5/0. Average score (2.125)
Win/lose count 3.5/0. Average score (2.4)
Win/lose count 1.5/0. Average score (2.25)
Win/lose count 2.0/0. Average score (2.2142857142857144)
Win/lose count 0.5/0. Average score (2.0)
Win/lose count 1.0/0. Average score (1.8888888888888888)
Win/lose count 2.0/0. Average score (1.9)
Final score: 1.9
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Final score: 0.0


with this temperature, the results are worse for both the newtorks. Let's increase the temperature to 0.5

In [35]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 5.5/0. Average score (3.25)
Win/lose count 5.5/0. Average score (4.0)
Win/lose count 3.0/0. Average score (3.75)
Win/lose count 3.0/0. Average score (3.6)
Win/lose count 8.0/0. Average score (4.333333333333333)
Win/lose count 2.5/0. Average score (4.071428571428571)
Win/lose count 5.5/0. Average score (4.25)
Win/lose count 11.5/0. Average score (5.055555555555555)
Win/lose count 6.5/0. Average score (5.2)
Final score: 5.2
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/2.0. Average score (-0.75)
Win/lose count 2.0/2.0. Average score (-0.5)
Win/lose count 1.5/1.0. Average score (-0.25)
Win/lose count 1.0/2.0. Average score (-0.4)
Win/lose count 0/0. Average score (-0.3333333333333333)
Win/lose count 1.5/3.0. Average score (-0.5)
Win/lose count 0/1.0. Average score (-0.5625)
Win/lose count 0.5/3.0. Average score (-0.7777777777777778)
Win/lose count 0.5/0. Average score (-0.65)
Final score: 

Here also, the CNN outperform the FC network. With higher temperatures, the scores are better as there is more food on the game. the probleme is the same with all temperatures values : after some time, the network looks stuck on a side of the board and spends his turns alternating between several empty cells.

It's also really weird that the FC network is performing really bad when in training it was better than that. Anyway, we will continue with CNN networks

Another problem is the variability of the results, and the fact that the training is very very long... . These two facts prevent us from doing a good parameter optimization

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



we will implement a descreasing epsilon greedy exploration. we just add to the last functions a eps_decay argument, and we start with high values of epsilon to force the agent to explore all the cells.

In [0]:
def train_explore(agent,env,epoch,prefix='',eps_decay=0.1, verbose=True, plot=False):
    score = 0
    loss = 0
    scores, wins, losses = [], [], []

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-eps_decay))

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        scores.append(score)
        wins.append(win)
        losses.append(lose)

        if verbose : 
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

    if plot : 
      plt.plot(wins, label="wins")
      plt.plot(losses, label="losses")
      plt.xlabel("Epoch")
      plt.legend()
      plt.grid()


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #board with previously visited cells
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

with this strategy, the agent will first explore the environnment, then exploit it

In [16]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.6)
agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, 30, prefix='cnn_train_explore')

Epoch 000/030 | Loss 0.0191 | Win/lose count 23.5/19.80000000000003 (3.699999999999971)
Epoch 001/030 | Loss 0.0273 | Win/lose count 28.5/21.999999999999996 (6.5000000000000036)
Epoch 002/030 | Loss 0.0345 | Win/lose count 31.5/21.800000000000033 (9.699999999999967)
Epoch 003/030 | Loss 0.0340 | Win/lose count 29.0/19.600000000000026 (9.399999999999974)
Epoch 004/030 | Loss 0.0245 | Win/lose count 26.0/24.10000000000007 (1.899999999999931)
Epoch 005/030 | Loss 0.0300 | Win/lose count 35.0/20.50000000000003 (14.499999999999972)
Epoch 006/030 | Loss 0.0317 | Win/lose count 33.0/16.199999999999967 (16.800000000000033)
Epoch 007/030 | Loss 0.0300 | Win/lose count 28.5/13.69999999999997 (14.80000000000003)
Epoch 008/030 | Loss 0.0501 | Win/lose count 27.0/18.5 (8.5)
Epoch 009/030 | Loss 0.0268 | Win/lose count 19.5/21.300000000000033 (-1.8000000000000327)
Epoch 010/030 | Loss 0.0363 | Win/lose count 38.5/16.599999999999977 (21.900000000000023)
Epoch 011/030 | Loss 0.0160 | Win/lose count 27

In [18]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 11.0/0. Average score (11.0)
Win/lose count 30.0/2.0. Average score (19.5)
Win/lose count 45.5/2.0. Average score (27.5)
Win/lose count 20.5/3.0. Average score (25.0)
Win/lose count 11.0/2.0. Average score (21.8)
Win/lose count 12.0/0. Average score (20.166666666666668)
Win/lose count 2.5/0. Average score (17.642857142857142)
Win/lose count 11.0/0. Average score (16.8125)
Win/lose count 13.5/0. Average score (16.444444444444443)
Win/lose count 11.0/0. Average score (15.9)
Final score: 15.9


In [20]:
HTML(display_videos('cnn_test_explore9.mp4'))

The results are way better than previously. looking at the videos, we can see that the agents is indeed better, but the stucking problem is still there...

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***